In [1]:
#Função para extrair dados em formato CSV
def Extrair_Dados_Gerais(caminho=str,delimitador=';',pular_linha=None):
    '''Se trata de uma função para extrair dados de uma rede elétrica em CSV
    \n A função funciona com o uso de duas variáveis nos seguintes moldes:
    \n caminho=str
    \n delimitador=';'
    \n Retorna a média, desvio padrão, mediana, máximo, mínimo e data.
    '''
    
    import numpy as np
    import pandas as pd

    #Tentar ler o arquivo CSV no padrão usado pela empresa
    if pular_linha==None:
        dados=pd.read_csv(caminho,delimiter=delimitador,skiprows=pular_linha)
        #Renomeando coluna
        dados.columns=dados.iloc[1]
        #Excluindo linhas de texto
        dados.drop(dados.index[:2], inplace=True)
    #Se nessecitar pular linha, usar modelo diferente
    else:
        dados=pd.read_csv(caminho,skiprows=pular_linha)
        #Renomeando coluna
        dados.columns=dados.iloc[0]
        #Excluindo linhas de texto
        dados.drop(dados.index[:1], inplace=True)
    #Resetando Indice
    dados.reset_index(inplace=True,drop=True)
    #Salvando data
    data=dados['Date'][1]
    #Excluindo colunas não númericas
    dados.drop(columns=['Date','Time','PDmP','PDmIA_D/T','PDmIB_D/T','PDmIC_D/T','PDmIAVG_D/T','PDmP_D/T','PDmQ_D/T','PDmS_D/T'],inplace=True)
    #Retirando dados entre pontos ABC e mantendo somente o consolidado
    #Sugiro esse processo para facilitar vizualização
    dados.drop(columns=['UA','UB','UC','UN','UTHDA','UTHDB','UTHDC','IA','IB','IC','ITHDA','ITHDB','ITHDC','FA','FB','FC','PFA','PFB','PFC','PA','PB','PC'],inplace=True)
    dados.drop(columns=['QA','QB','QC','SA','SB','SC','EPA','EPB','EPC','EQA','EQB','EQC','ESA','ESB','ESC','DmIA','DmIB','DmIC','PDmIA','PDmIB','PDmIC'],inplace=True)
    #Convertendo texto para número
    dados=dados.apply(pd.to_numeric)
    #Declarando variável
    consolidado=pd.DataFrame()
    #Calculando
    consolidado['Média']=pd.DataFrame(round(dados.mean(),1))
    consolidado['Desvio padrão']=pd.DataFrame(round(dados.std(),1))
    consolidado['Mediana']=pd.DataFrame(round(dados.median(),1))
    consolidado['Máximo']=pd.DataFrame(round(dados.max(),1))
    consolidado['Mínimo']=pd.DataFrame(round(dados.min(),1))
    #Trazendo Data
    consolidado['Data']=[data]*len(consolidado)
   
    return consolidado

In [2]:
#Função para formar relatório em HTML
def Relatorio_HTML(*args):
    '''Se trata de uma função para transformar dataframes em uma tabela HTML
    \n A função funciona com a entrada de n dataframes extraidos:
    '''
    import pandas as pd
    
    #Concatenando
    Relatorio=pd.concat(args)
    #Recriando indice para melhor organização
    Relatorio['Indice']=Relatorio.index
    Relatorio.reset_index(inplace=True,drop=True)
    Relatorio=Relatorio.set_index(['Data','Indice'])
    #Formando tabela em HTML
    Relatorio=Relatorio.to_html(buf='C:/Users/andre/OneDrive/Área de Trabalho/Relatorio.html')
    return Relatorio

In [4]:
#Extraindo dados das planilhas CSV
Geral328=Extrair_Dados_Gerais(caminho='D:/Downloads2/Dados_compilado/DataSheet_3421198001_3P4W_20220328.csv')
Geral329=Extrair_Dados_Gerais(caminho='D:/Downloads2/Dados_compilado/DataSheet_3421198001_3P4W_20220329.csv')
Geral330=Extrair_Dados_Gerais(caminho='D:/Downloads2/Dados_compilado/DataSheet_3421198001_3P4W_20220330.csv')
Geral331=Extrair_Dados_Gerais(caminho='D:/Downloads2/Dados_compilado/DataSheet_3421198001_3P4W_20220331.csv')
Geral401=Extrair_Dados_Gerais(caminho='D:/Downloads2/Dados_compilado/DataSheet_3421198001_3P4W_20220401.csv')
Geral402=Extrair_Dados_Gerais(caminho='D:/Downloads2/Dados_compilado/DataSheet_3421198001_3P4W_20220402.csv')
Geral403=Extrair_Dados_Gerais(pular_linha=1,delimitador=None,caminho='D:/Downloads2/Dados_compilado/DataSheet_3421198001_3P4W_20220403.csv')
#Formar relatório
Relatorio=Relatorio_HTML(Geral328,Geral329,Geral330,Geral331,Geral401,Geral402)